In [1]:
from sympy import cacheit, Symbol
from sympy.core.function import Function, UndefinedFunction, Application

class FOLConstant(Symbol):
    is_Constant = True

class FOLFunction(Function):
    def __new__(cls, *args, **options):
        if cls is FOLFunction:
            options['bases'] = (AppliedFOLFunction,)
            # a predicate is considered as an UndefinedFunction
            res = UndefinedFunction(*args, **options)
            return res
        return super(FOLFunction, cls).__new__(cls, *args, **options)

    def ground(self, var_name, constant):
        fun = FOLFunction(self.name)
        variables = []
        for i in range(0, len(self.args)):
          if self.args[i].is_Symbol and self.args[i].name == var_name:
            variables.append(constant)
          elif self.args[i].is_Symbol:
            variables.append(self.args[i])
          else:
            variables.append(self.args[i].ground(var_name, constant))
        return fun(*variables)

class Formula(Function):
    is_Predicate = False
    is_Quantifier = False

    # called when operation is not allowed (as in sympy library)
    def _noop(self, other=None):
        raise TypeError('First order logic term not allowed in this context.')

    # redefinition of and
    def __and__(self, other):
        """Overloading for & operator"""
        return FOLAnd(self, other)

    __rand__ = __and__

    # redefinition of or
    def __or__(self, other):
        """Overloading for |"""
        return FOLOr(self, other)

    __ror__ = __or__

    # redefinition of not
    def __invert__(self):
        """Overloading for ~"""
        return FOLNot(self)

    # redefinition of implies
    def __rshift__(self, other):
        """Overloading for >>"""
        return FOLImplies(self, other)

    # redefinition of is implied
    def __lshift__(self, other):
        """Overloading for <<"""
        return FOLImplies(other, self)

    __rrshift__ = __lshift__
    __rlshift__ = __rshift__

    # redefinition of is xor
    def __xor__(self, other):
        return FOLOr(FOLAnd(self, FOLNot(other)), FOLAnd(FOLNot(self), other))

    __rxor__ = __xor__

    __add__ = _noop
    __radd__ = _noop
    __sub__ = _noop
    __rsub__ = _noop
    __mul__ = _noop
    __rmul__ = _noop
    __pow__ = _noop
    __rpow__ = _noop
    __rdiv__ = _noop
    __truediv__ = _noop
    __div__ = _noop
    __rtruediv__ = _noop
    __mod__ = _noop
    __rmod__ = _noop
    _eval_power = _noop

    def _apply_not(self):
        raise NotImplementedError("Implementation Error")

    # negation normal form
    def to_nnf(self):
        raise NotImplementedError("Implementation Error")

    # skolem normal form
    def to_snf(self, skolem_function_symbol= None):
        from sympy.first_order_logic.algorithm import to_snf
        return to_snf(self, skolem_function_symbol)

    # prenex normal form
    def to_pnf(self):
        from sympy.first_order_logic.algorithm import to_pnf
        return to_pnf(self)

    def to_list_clauses(self):
        from sympy.first_order_logic.algorithm import to_list_clauses
        return to_list_clauses(self)

    def is_satisfiable(self, algorithm=None):
        f = self.to_nnf()
        f = f.to_pnf()
        f = f.to_snf()
        if algorithm is None:
            from sympy.first_order_logic.algorithm import Resolution
            algorithm = Resolution()
        c = f.to_list_clauses()
        return algorithm(*c)

"""
def func(self):
        return self.__class__
"""
class FOLLogicOperator(Formula):
    def to_nnf(self):
        # every argument of the formula must have its own to_nnf() method
        return self.func(*[a.to_nnf() for a in self.args])


class FOLAnd(FOLLogicOperator):
    nargs = 2

    # not(A and B) = not A or not B
    def _apply_not(self):
        return FOLOr(self.args[0]._apply_not(), self.args[1]._apply_not())
    
    def _latex(self,printer,*args):        
        if self.args[0].is_Predicate or self.args[0].is_Quantifier or self.args[0].func == FOLNot:
            if self.args[1].is_Predicate or self.args[1].is_Quantifier or self.args[1].func == FOLNot:
                return r'%s \wedge %s'%tuple([self.args[0]._latex(printer,*args), self.args[1]._latex(printer,*args)])
            else:
                return r'%s \wedge (%s)'%tuple([self.args[0]._latex(printer,*args), self.args[1]._latex(printer,*args)])
        elif self.args[1].is_Predicate or self.args[1].is_Quantifier or self.args[1].func == FOLNot:
            return r'(%s) \wedge %s'%tuple([self.args[0]._latex(printer,*args), self.args[1]._latex(printer,*args)])

        return r'(%s) \wedge (%s)'%tuple([self.args[0]._latex(printer,*args), self.args[1]._latex(printer,*args)])
    
    def ground(self, var_name, constant):
        return FOLAnd(self.args[0].ground(var_name, constant),self.args[1].ground(var_name, constant))


class FOLOr(FOLLogicOperator):
    nargs = 2
    
    # not(A or B) = not A and not B
    def _apply_not(self):
        return FOLAnd(self.args[0]._apply_not(), self.args[1]._apply_not())
    
    def _latex(self,printer,*args):
        if self.args[0].is_Predicate or self.args[0].is_Quantifier or self.args[0].func == FOLNot:
            if self.args[1].is_Predicate or self.args[1].is_Quantifier or self.args[1].func == FOLNot:
                return r'%s \vee %s'%tuple([self.args[0]._latex(printer,*args), self.args[1]._latex(printer,*args)])
            else:
                return r'%s \vee (%s)'%tuple([self.args[0]._latex(printer,*args), self.args[1]._latex(printer,*args)])
        elif self.args[1].is_Predicate or self.args[1].is_Quantifier or self.args[1].func == FOLNot:
            return r'(%s) \vee %s'%tuple([self.args[0]._latex(printer,*args), self.args[1]._latex(printer,*args)])

        return r'(%s) \vee (%s)'%tuple([self.args[0]._latex(printer,*args), self.args[1]._latex(printer,*args)])

    def ground(self, var_name, constant):
        return FOLOr(self.args[0].ground(var_name, constant),self.args[1].ground(var_name, constant))
    
    
class FOLNot(FOLLogicOperator):
    nargs = 1

    # in NNF the logic operator not must only be in front of predicates
    def to_nnf(self):
        expr = self.args[0]
        if (expr.is_Predicate):
            return self

        res = self.args[0]._apply_not()
        return res.to_nnf()
    
    # not (not A) = A
    def _apply_not(self):
        return self.args[0]

    def _latex(self,printer,*args):
        if self.args[0].is_Predicate or self.args[0].is_Quantifier or self.args[0].func == FOLNot:
            return r'\lnot %s'%tuple([self.args[0]._latex(printer,*args)])
    
        return r'\lnot (%s)'%tuple([self.args[0]._latex(printer,*args)])

    def ground(self, var_name, constant):
        return FOLNot(self.args[0].ground(var_name, constant))


class FOLImplies(FOLLogicOperator):
    nargs = 2

    # A -> B = not A or B
    def to_nnf(self):
        return FOLOr(self.args[0]._apply_not().to_nnf(), self.args[1].to_nnf())

    # not (A -> B) = A and not B
    def _apply_not(self):
        return FOLAnd(self.args[0], self.args[1]._apply_not())
    
    def _latex(self,printer,*args):
        if self.args[0].is_Predicate or self.args[0].is_Quantifier or self.args[0].func == FOLNot:
            if self.args[1].is_Predicate or self.args[1].is_Quantifier or self.args[1].func == FOLNot:
                return r'%s \rightarrow %s'%tuple([self.args[0]._latex(printer,*args), self.args[1]._latex(printer,*args)])
            else:
                return r'%s \rightarrow (%s)'%tuple([self.args[0]._latex(printer,*args), self.args[1]._latex(printer,*args)])
        elif self.args[1].is_Predicate or self.args[1].is_Quantifier or self.args[1].func == FOLNot:
            return r'(%s) \rightarrow %s'%tuple([self.args[0]._latex(printer,*args), self.args[1]._latex(printer,*args)])

        return r'(%s) \rightarrow (%s)'%tuple([self.args[0]._latex(printer,*args), self.args[1]._latex(printer,*args)])

    def ground(self, var_name, constant):
        return FOLImplies(self.args[0].ground(var_name, constant),self.args[1].ground(var_name, constant))


class Predicate(Formula):
    is_Atom = True
    is_Predicate = True
    is_Equality = False

    @cacheit
    def __new__(cls, *args, **options):
        if cls is Predicate:
            options['bases'] = (AppliedPredicate,)
            # a predicate is considered as an UndefinedFunction
            res = UndefinedFunction(*args, **options)
            return res
        return super(Predicate, cls).__new__(cls, *args, **options)

    def to_nnf(self):
        return self

    def _apply_not(self):
        return FOLNot(self)

    def _latex(self,printer,*args):
        return r'%s'% str(self)
    
    def ground(self, var_name, constant):
        vec = []
        for i in range(0,len(self.args)):
          if self.args[i].name == var_name:
            vec.append(i)
        pred = Predicate(self.name)

        variables = []
        for i in range(0,len(self.args)):
          if i in vec:
            variables.append(constant)
          else:
            if self.args[i].is_Symbol:
              variables.append(self.args[i])
            else:
              variables.append(self.args[i].ground(var_name,constant))
        return pred(*variables)



class FOLEquality(Predicate):
    is_Equality = True
    nargs = 2

class AppliedPredicate(Predicate):
    pass

class AppliedFOLFunction(FOLFunction):
    pass

class Quantifier(Formula):
    nargs = 1
    is_Quantifier = True
    is_Forall = False
    is_Exists = False
    _symbol = None

    def __new__(cls, *args, **options):
        if cls in (Forall, Exists):
            argslist = list(args)
            symbol = argslist[0] # variable which is not free
            argslist[0] = cls.name + symbol.name
            args = tuple(argslist)
            options['bases'] = (cls._getAppliedClass(),)
            res = UndefinedFunction(*args, **options)
            res._symbol = symbol
            return res

        return super(Quantifier, cls).__new__(cls, *args, **options)

    @property
    def symbol(self):
        return self._symbol

    @property
    def formula(self):
        return self.args[0]

    def to_nnf(self):
        return self.func(self.formula.to_nnf())


class Forall(Quantifier):
    is_Forall = True
    name = 'Forall_'
    
    # not forall = exist not
    def _apply_not(self):
        exists = Exists(self.symbol)
        return exists(self.formula._apply_not())

    @staticmethod
    def _getAppliedClass():
        return AppliedForall
    
    def _latex(self,printer,*args):
        if self.formula.is_Predicate or self.args[0].func == FOLNot:
            return r'\forall %s %s'%tuple([printer._print(self.symbol,*args), self.formula._latex(printer,*args)])

        return r'\forall %s (%s)'%tuple([printer._print(self.symbol,*args), self.formula._latex(printer,*args)])
    
    def ground(self, set_const):
        formula = self.formula.ground(self.symbol.name, set_const[0])
        for i in range(1, len(set_const)):
          formula = FOLAnd(formula, self.formula.ground(self.symbol.name, set_const[i]))
        return formula

class AppliedForall(Forall):
    pass


class Exists(Quantifier):
    is_Exists = True
    name = 'Exists_'

    # not exists = forall not
    def _apply_not(self):
        forall = Forall(self.symbol)
        return forall(self.formula._apply_not())

    @staticmethod
    def _getAppliedClass():
        return AppliedExists
    
    def _latex(self,printer,*args):
        if self.formula.is_Predicate or self.args[0].func == FOLNot:
            return r'\exists %s %s'%tuple([printer._print(self.symbol,*args), self.formula._latex(printer,*args)])

        return r'\exists %s (%s)'%tuple([printer._print(self.symbol,*args), self.formula._latex(printer,*args)])
    
    def ground(self, set_const):
        formula = self.formula.ground(self.symbol.name, set_const[0])
        for i in range(1, len(set_const)):
          formula = FOLOr(formula, self.formula.ground(self.symbol.name, set_const[i]))
        return formula
    


class AppliedExists(Exists):
    pass

def find_quantifier(formula):
  if formula.is_Predicate:
    return False
  if formula.is_Quantifier:
    return True
  if formula.func == FOLNot:
    return find_quantifier(formula.args[0])
  else:
    return find_quantifier(formula.args[0]) or find_quantifier(formula.args[1])


def grounding(formula, constants_set):
    # assume that the formula is in NNF
    '''
    find = find_Quantifier(formula.formula)
    if formula.is_Quantifier and not find:
      return formula.ground(constants_set)
    if find:
      return grounding(formula., constants_set)

    arg = formula.args[i]
    while not arg.is_Quantifier:
        if arg.func == Forall:
            # arg.symbol -> variable
            # arg.formula -> formula
            grounds = arg.formula.xreplace({arg.symbol: constants_set[0]})
            for constant in constants_set[1:]:
                temp = arg.formula.xreplace({arg.symbol: constant})
                grounds = FOLAnd(grounds, temp)
        elif arg.func == Exists:
            grounds = arg.formula.xreplace({arg.symbol: constants_set[0]})
            for constant in constants_set[1:]:
                temp = arg.formula.xreplace({arg.symbol: constant})
                grounds = FOLOr(grounds, temp)
        i += 1
        arg = formula.args[i]
   '''


In [2]:
from sympy.abc import x, y, z
from sympy.core import Function, Symbol

a = FOLConstant('a')
b = FOLConstant('b')
c = FOLConstant('c')
d = FOLConstant('d')

f = Function('f')
g = Function('g')
h = Function('h')

P = Predicate('P')
Q = Predicate('Q')
R = Predicate('R')

# print examples:
Q(x) & P(y)
Q(x) & Exists(x) (P(x,y))
Q(x) & Forall(x) (P(x,y))
Exists(x) (P(x,y)) & Q(x) 
Forall(x) (P(x,y)) & Q(x) 
Exists(x) (P(x,y)) & Exists(x) (P(x,y)) 
Exists(x) (f(x,y)) & Forall(x) (P(x,y))
Forall(x) (f(x,y)) & Exists(x) (P(x,y)) 
Forall(x) (P(x,y)) & Forall(x) (P(x,y)) 

Q(x) | P(y)
Q(x) | Exists(x) (P(x,y))
Q(x) | Forall(x) (P(x,y))
Exists(x) (P(x,y)) | Q(x) 
Forall(x) (P(x,y)) | Q(x) 
Exists(x) (P(x,y)) | Exists(x) (f(x,y)) 
Exists(x) (P(x,y)) | Forall(x) (f(x,y))
Forall(x) (P(x,y)) | Exists(x) (f(x,y)) 
Forall(x) (P(x,y)) | Forall(x) (f(x,y)) 
 
~Q(x)
~Exists(x) (P(x,y))
~Forall(x) (P(x,y))
~Forall(x) (~Forall(x) (P(x,y)))
~Forall(x) (~Forall(x) (~P(x,y)>> ~P(x)))

# NNF (call to_nnf before grounding):
form = ~Forall(x) (P(y) & ((~P(x,y) & Q(f(z))) >> Exists (x) (~P(x) & Q(f(x),a,z))))
form.to_nnf()


True

In [3]:
ciao = P(x,y,x)
var_name = 'x'
set_const = [a,b,c,d]
vec = []
for i in range(0,len(ciao.args)):
  
  if ciao.args[i].name == var_name:
    vec.append(i)
pred = Predicate(ciao.name)

variables = []
for i in range(0,len(ciao.args)):
  if i in vec:
    variables.append(set_const[0])
  else:
    variables.append(ciao.args[i])
formula = pred(*variables)

for j in range(1, len(set_const)):
  variables = []
  for i in range(0,len(ciao.args)):
    if i in vec:
      variables.append(set_const[j])
    else:
      variables.append(ciao.args[i])
  formula = FOLAnd(formula, pred(*variables))




In [7]:
f = FOLFunction('f')
g = FOLFunction('g')
h = FOLFunction('h')

P = Predicate('P')

ciao = P(f(g(h(x,y),f(x,x)),z,x)) & Q(x) | (P(x) & Q(x))
find_quantifier(ciao)


False